**Welcome to the example simple robot agent Colab from Mbodi!**

This colab a minimal example demonstrating how to use mbodied-agents for a simple intelligent robot agent and output actions based on the observation and human instruction.

In [1]:
#@title [!Important]Please use a GPU runtime.
!nvidia-smi

Wed Jun  5 23:37:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   57C    P8              13W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#Mbodied Agents Setup

## Clone the Mbodied Agents Repo


In [2]:
!git clone https://github.com/MbodiAI/mbodied-agents.git
%cd mbodied-agents

Cloning into 'mbodied-agents'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 232 (delta 38), reused 25 (delta 25), pack-reused 168
Receiving objects: 100% (232/232), 47.13 MiB | 10.92 MiB/s, done.
Resolving deltas: 100% (105/105), done.
/content/mbodied-agents


## Install Mbodied Agents Dependencies

In [3]:
!pip install coverage[toml]
!pip install toml
!pip install gym
!pip install pillow
!pip install numpy
!pip install opencv-python
!pip install anthropic
!pip install openai
!pip install backoff
!pip install pydantic
!pip install pydantic_numpy
!pip install torch
!pip install h5py
!pip install datasets
!pip install art
!pip install jsonref
# Build the mbodied-agents package.
!pip install -e . --no-deps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.6/231.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.7/862.7 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using

# SimplerEnv Setup

In [4]:
# @title Install vulkan for rendering
!apt-get install -yqq --no-install-recommends libvulkan-dev vulkan-tools
# below fixes some bugs introduced by some recent Colab changes
!mkdir -p /usr/share/vulkan/icd.d
!wget -q -P /usr/share/vulkan/icd.d https://raw.githubusercontent.com/haosulab/ManiSkill/main/docker/nvidia_icd.json
!wget -q -O /usr/share/glvnd/egl_vendor.d/10_nvidia.json https://raw.githubusercontent.com/haosulab/ManiSkill/main/docker/10_nvidia.json

Selecting previously unselected package libvulkan1:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libvulkan1_1.3.204.1-2_amd64.deb ...
Unpacking libvulkan1:amd64 (1.3.204.1-2) ...
Selecting previously unselected package libvulkan-dev:amd64.
Preparing to unpack .../libvulkan-dev_1.3.204.1-2_amd64.deb ...
Unpacking libvulkan-dev:amd64 (1.3.204.1-2) ...
Selecting previously unselected package vulkan-tools.
Preparing to unpack .../vulkan-tools_1.3.204.0+dfsg1-1_amd64.deb ...
Unpacking vulkan-tools (1.3.204.0+dfsg1-1) ...
Setting up libvulkan1:amd64 (1.3.204.1-2) ...
Setting up libvulkan-dev:amd64 (1.3.204.1-2) ...
Setting up vulkan-tools (1.3.204.0+dfsg1-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is n

In [5]:
# @title Make sure vulkan is installed correctly
!vulkaninfo | head -n 5

'DISPLAY' environment variable not set... skipping surface info
error: XDG_RUNTIME_DIR not set in the environment.
VULKANINFO

Vulkan Instance Version: 1.3.204


In [6]:
# @title Install Real2Sim
!git clone https://github.com/simpler-env/ManiSkill2_real2sim.git
!pip install -e ./ManiSkill2_real2sim
!git clone https://github.com/simpler-env/SimplerEnv.git
!pip install -e ./SimplerEnv
!mkdir ./SimplerEnv/checkpoints

Cloning into 'ManiSkill2_real2sim'...
remote: Enumerating objects: 3448, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 3448 (delta 73), reused 96 (delta 34), pack-reused 3301
Receiving objects: 100% (3448/3448), 188.75 MiB | 14.16 MiB/s, done.
Resolving deltas: 100% (1833/1833), done.
Updating files: 100% (661/661), done.
Obtaining file:///content/mbodied-agents/ManiSkill2_real2sim
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
# @title Install other requirements
!pip install --quiet mediapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.7 MB/s eta 0:00:00


In [8]:
# @title [Important]Post Installation

# run this so local pip installs are recognized
import site
site.main()

## Import all neccessary packages

Note that we are not importing the audio handler here in the Colab. We are going to use keyboard and screen for IO.

In [9]:
import os
import logging

import click
from pydantic import Field
from pydantic_core import from_json
from gym import spaces

import sys
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), 'src'))

from mbodied_agents.agents.language import CognitiveAgent
# For this colab example, we are not using audio IO.
# from mbodied_agents.agents.sense.audio_handler import AudioHandler
from mbodied_agents.base.sample import Sample
from mbodied_agents.hardware.sim_interface import SimInterface
from mbodied_agents.types.controls import HandControl
from mbodied_agents.types.vision import Image
from mbodied_agents.data.recording import Recorder

## Define a Sample for your robot agent's output.

This can be customized to be anything you like. In this example, we have an answer field, which is what the robot speaks to you, and a list of hand controls (X,Y,Z,R,P,Y) which is the end effector delta for robot's hand. Sample can be converted to gym space easily.

In [10]:
class AnswerAndActionsList(Sample):
    """A customized pydantic type for robot's reply and actions."""
    answer: str | None = Field(
        default="",
        description="Short, one sentence answer to user's question or request.",
    )
    actions: list[HandControl] | None = Field(
        default=[],
        description="List of actions to be taken by the robot.",
    )


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Initialize the Cognitive Agent

Cognitive Agent is the entry point for your intelligent robot. It's responsible of handling the planning, reasoning, or action output. You may even have multiple Cognitive Agent in parallel handling different aspect for your robot, i.e. planning, motion control, sensory, etc.

For the simplest example, we can use OpenAI's GPT for the cognitive Agent here. Anthropic is also supported here.

Mbodi's backend and HuggingFace backend is also upcoming.

The system prompt is minimal. You can provide more context if you like.

In [11]:
openai_api_key = "your openai api key here"

SYSTEM_PROMPT = f"""
    You are robot with vision capabilities.
    For each task given, you respond in JSON format. Here's the JSON schema:
    {AnswerAndActionsList.model_json_schema()}
    """

robot_agent = CognitiveAgent(context=SYSTEM_PROMPT, api_service="openai", api_key=openai_api_key)

# No audio for this Colab
# audio = AudioHandler(use_pyaudio=False)

Welcome to
 __  __  _                 _  _ 
|  \/  || |__    ___    __| |(_)
| |\/| || '_ \  / _ \  / _` || |
| |  | || |_) || (_) || (_| || |
|_|  |_||_.__/  \___/  \__,_||_|
                                

A platform for intelligent embodied agents.


Initializing cognitive agent for robot using backend: openai


## Define data Recorder

Initialize data recorder for every conversation and every action.
A new dataset is created each time you interact, teach or train the robot.
Available for further augmentation finetuning and model training.

Note that we use the gym space here. You can also customize this to be type/format you like.


In [12]:
observation_space = spaces.Dict({
    'image': Image(size=(224, 224)).space(),
    'instruction': spaces.Text(1000)
})
action_space = AnswerAndActionsList(actions=[HandControl()] * 6).space()
recorder = Recorder('example_recorder',
                    out_dir='saved_datasets',
                    observation_space=observation_space,
                    action_space=action_space)


Initializing dataset recorder...
Recording dataset to saved_datasets/example_recorder.h5
To learn more about how to use this dataset, augment the data, train/finetune a foundational model, please go to: https://forms.gle/rv5rovK93dLucma37



/usr/local/lib/python3.10/dist-packages/gym/utils/seeding.py:63: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


## Initializing and Resetting the Robotic Task Environment

In [18]:
import simpler_env
from simpler_env.utils.env.observation_utils import get_image_from_maniskill2_obs_dict
import mediapy
import sapien.core as sapien
import numpy as np

task_name = "google_robot_pick_coke_can"  # @param ["google_robot_pick_coke_can", "google_robot_move_near", "google_robot_open_drawer", "google_robot_close_drawer", "widowx_spoon_on_towel", "widowx_carrot_on_plate", "widowx_stack_cube", "widowx_put_eggplant_in_basket"]

if 'env' in locals():
  print("Closing existing env")
  env.close()
  del env
env = simpler_env.make(task_name)

# Note: we turned off the denoiser as the colab kernel will crash if it's turned on
# To use the denoiser, please git clone our SIMPLER environments
# and perform evaluations locally.
sapien.render_config.rt_use_denoiser = False

obs, reset_info = env.reset()
print("Reset info", reset_info)


Closing existing env
Reset info {'scene_name': 'google_pick_coke_can_1_v4', 'scene_offset': None, 'scene_pose': None, 'scene_table_height': 0.87, 'urdf_version': 'recolor_tabletop_visual_matching_1', 'rgb_overlay_path': '/content/mbodied-agents/ManiSkill2_real2sim/data/real_inpainting/google_coke_can_real_eval_1.png', 'rgb_overlay_cameras': ['overhead_camera'], 'rgb_overlay_mode': 'background', 'disable_bad_material': False, 'model_id': 'opened_coke_can', 'model_scale': 1.0, 'distractor_model_ids': None, 'distractor_model_scales': None, 'obj_init_pose_wrt_robot_base': Pose([0.587925, -0.0238302, 0.840576], [0.707052, -0.0081018, -0.01162, -0.70702]), 'orientation': 'laid_vertically'}


## Simulating Cognitive Agent Interaction with SimplerEnv
In this section, we'll demonstrate how to interact with the SimplerEnv environment using a cognitive agent. The agent will receive instructions, process them, and execute corresponding actions within the simulation. This example uses a simple loop to simulate the agent's decision-making process and visualize the results.

In [19]:
done, truncated = False, False

#
timestep = 0
num_iteration = 5

image = get_image_from_maniskill2_obs_dict(env, obs)
images = [image]

while not (done or truncated):
    # instruction = audio.listen()
    # instruction = input("Please type your instruction: ")
    instruction = "Move Left"
    print("[YOU SAID]:", instruction)
    if instruction == "exit":
      break

    # Get response from the cognitive agent based on the instruction and image
    response = robot_agent.act(instruction, image)[0]
    response = response.replace("```json", "").replace("```", "")
    print("Response:", response)

    # Validate the response to the pydantic object.
    answer_actions = AnswerAndActionsList.model_validate(from_json(response))

    # Let the robot speak.
    if answer_actions.answer:
        # audio.speak(answer_actions.answer)
        print("[ROBOT SPEAKS]:", answer_actions.answer)

    # Execute the actions with the SimplerEnv Interface
    if answer_actions.actions:
        for action in answer_actions.actions:
            obs, reward, done, truncated, info = env.step(np.array(action.flatten('list')))

    print(timestep, info)
    # update image observation
    image = get_image_from_maniskill2_obs_dict(env, obs)
    images.append(image)
    timestep += 1

    # Record the dataset for training.
    recorder.record(observation={
        'image': Image(image, size=(224,224)),
        'instruction': instruction,
    }, action=answer_actions)

    if timestep > num_iteration:
      done = True

episode_stats = info.get('episode_stats', {})
print("Episode stats", episode_stats)
mediapy.show_video(images, fps=10)

[YOU SAID]: Move Left
Response: 
{
  "answer": "Moving left.",
  "actions": [
    {
      "pose": {
        "x": 0,
        "y": 0.5,
        "z": 0,
        "roll": 0,
        "pitch": 0,
        "yaw": 0
      },
      "grasp": {
        "value": 1
      }
    }
  ]
}

[ROBOT SPEAKS]: Moving left.
0 {'elapsed_steps': 1, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success': False, 'episode_stats': OrderedDict([('n_lift_significant', 0), ('consec_grasp', False), ('grasped', False)])}
[YOU SAID]: Move Left
Response: 
{
  "answer": "Moving left.",
  "actions": [
    {
      "pose": {
        "x": 0,
        "y": 0.5,
        "z": 0,
        "roll": 0,
        "pitch": 0,
        "yaw": 0
      },
      "grasp": {
        "value": 1
      }
    }
  ]
}

[ROBOT SPEAKS]: Moving left.
1 {'elapsed_steps': 2, 'is_grasped': False, 'consecutive_grasp': False, 'lifted_object': False, 'lifted_object_significantly': False, 'success

This browser does not support the video tag.
